# Time torch vs numpy

## Setup 

activate Spikeinterf..

In [2]:
import torch
import numpy as np

In [2]:
# # takes 1 min
# a = torch.normal(2, 3, size=(40000 * 600, 384))
# a_array = a.cpu().detach().numpy()

# # takes 7 min
# b = np.random.normal(2, 3, size=(40000 * 600, 384))

### Efficiently add independent noise to an array

* noise has RMS of 2 uV
* 40 min at 40,000 Hz (40 * 60 * 40,0000)
* warning: the node might not have sufficient memory to allocate, you can reduce n_sites

In [4]:
# (6m)
n_sites = 384
sfreq = 40000
t_secs = 40 * 60  # 40 mins
n_samples = t_secs * sfreq
gain = 1000

# simulate toy voltage signal
device = torch.device("cuda:0")
signal = torch.normal(0, 3, size=(n_samples, n_sites))  # too large for GPU memory

* numpy solution is too slow (past 1 hour - 2h30)

In [7]:
# # 74 mins and crashes ...
# # noise
# noise = np.random.normal(
#     0,
#     2,
#     [384, n_samples],
# )

# # scale traces and add missing noise
# noised = signal.T * gain + noise

* torch solution is much faster (10 mins)

In [42]:
# 10m
n_site = 384

# zero - mean
mean = torch.tensor([0], dtype=torch.float32)

# missing noise stds
l1_noise = torch.tensor(2.0).repeat(76)
l23_noise = torch.tensor(3.0).repeat(76)
l4_noise = torch.tensor(4.0).repeat(76)
l5_noise = torch.tensor(5.0).repeat(76)
l6_noise = torch.tensor(6.0).repeat(80)
noise_std = torch.cat((l1_noise, l23_noise, l4_noise, l5_noise, l6_noise), dim=0)

# create missing noise matrix
missing_noise = torch.randn(n_samples, n_site) * noise_std + mean

# add noise
noisy = (signal.T * gain) + missing_noise.T

# cast as numpy array
noisy = noisy.cpu().detach().numpy()

In [5]:
# toy traces
traces = torch.normal(0, 3, size=(n_samples, n_sites))  # too large for GPU memory
traces = traces.cpu().detach().numpy()

In [8]:
import os

# get config
NOISE_PATH = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/dataeng/0_silico/neuropixels_lfp_10m_384ch_hex0_40Khz_2023_10_18/be011315-9555-493e-a59c-27f42d1058ed/campaign/preprocessed/missing_noise_"

# if already fitted data exist
# return concatenated noise per layer
noises = ()
if os.path.isfile(NOISE_PATH + "L1.npy"):
    noises += (np.load(NOISE_PATH + "L1.npy", allow_pickle=True).item(),)
if os.path.isfile(NOISE_PATH + "L2_3.npy"):
    noises += (np.load(NOISE_PATH + "L2_3.npy", allow_pickle=True).item(),)
if os.path.isfile(NOISE_PATH + "L4.npy"):
    noises += (np.load(NOISE_PATH + "L4.npy", allow_pickle=True).item(),)
if os.path.isfile(NOISE_PATH + "L5.npy"):
    noises += (np.load(NOISE_PATH + "L5.npy", allow_pickle=True).item(),)
if os.path.isfile(NOISE_PATH + "L6.npy"):
    noises += (np.load(NOISE_PATH + "L6.npy", allow_pickle=True).item(),)

In [ ]:
# import copy

# gain_prms = dict()
# gain_prms["gain_adjust"] = 0.9


# # get stored results for the best fit scaling factor and missing noise
# # set seed for reproducibility
# np.random.seed(noises[0]["seed"])

# # make writable (40s/h recording)
# fitted_traces = copy.copy(traces).T
# nsites = traces.shape[1]
# ntimepoints = traces.shape[0]

# # - scale trace and add missing noise to each site
# for ix, _ in enumerate(noises):

#     # get sites, scaling factor and missing noise
#     sites = noises[ix]["layer_sites_ix"]
#     gain = noises[ix]["gain"] * gain_prms["gain_adjust"]  # adjust gain
#     missing_noise = noises[ix]["missing_noise_rms"]

#     # reconstruct fitted missing noise traces
#     missing_noise_traces = np.random.normal(
#         0,
#         missing_noise,
#         [nsites, ntimepoints],
#     )

#     # scale traces and add missing noise
#     fitted_traces[sites, :] = traces[:, sites].T * gain + missing_noise_traces[sites, :]

#     # release memory
#     del missing_noise_traces

In [11]:
def create_noise_matrix(noises, n_sites, n_samples):

    # set reproducibility
    torch.manual_seed(noises[0]["seed"])
    np.random.seed(noises[0]["seed"])

    # assign noise rms to each site
    # - zeros will be added to sites outside cortex (mean=0, std=0)
    # - noise_rms is a column-vector of n sites
    noise_rms = torch.tensor(0).repeat(n_sites, 1)
    for ix, _ in enumerate(noises):
        noise_rms[noises[ix]["layer_sites_ix"]] = noises[ix]["missing_noise_rms"]

    # unit-test
    assert all(np.isnan(noise_rms)) == False, "there should be no nan values"
    return torch.randn(n_sites, n_samples) * noise_rms

In [19]:
traces = torch.from_numpy(traces)

In [12]:
# create noise tensor
n_sites = traces.shape[1]
missing_noise = create_noise_matrix(noises, n_sites, n_samples)

In [20]:
# add noise and cast as array
noisy = (traces.T * gain) + missing_noise
noisy = noisy.cpu().detach().numpy()

In [18]:
missing_noise

tensor([[-0., -0., 0.,  ..., 0., 0., 0.],
        [-0., -0., 0.,  ..., -0., -0., 0.],
        [0., -0., -0.,  ..., 0., 0., -0.],
        ...,
        [-0., 0., -0.,  ..., -0., 0., 0.],
        [-0., -0., -0.,  ..., -0., -0., -0.],
        [-0., 0., -0.,  ..., 0., 0., 0.]])